In [1]:
import os
import pandas as pd
import numpy as np
import cv2

from matplotlib import pyplot as plt
import matplotlib
%matplotlib inline

In [2]:
# DATA_ROOT = '../data/'

# df_train = pd.read_csv(os.path.join(DATA_ROOT,'train_proc_v2.csv'),index_col='id')
# dist = []
# mask_pixels = []
# masks = []
# for id in df_train.index:
#     img = cv2.imread(os.path.join('../data/train/images/','{}.png'.format(id)), cv2.IMREAD_GRAYSCALE)
#     mask = cv2.imread(os.path.join('../data/train/masks/','{}.png'.format(id)), cv2.IMREAD_GRAYSCALE)
#     dist.append(np.unique(img).shape[0])
#     mask_pixels.append((mask/255).sum())
#     masks.append(np.array(mask)/255.)
# df_train['unique_pixels'] = dist
# df_train['mask_pixels'] = mask_pixels
# df_train['masks'] = masks
# df_train[['fold','unique_pixels','coverage_class']].to_csv(os.path.join(DATA_ROOT,'train_proc_v2_gr.csv'))

In [3]:
DATA_ROOT = '../data/'

df_train = pd.read_csv(os.path.join(DATA_ROOT,'train.csv'),index_col='id')
depths = pd.read_csv(os.path.join(DATA_ROOT,'depths.csv'),index_col='id')
df_train = df_train.join(depths)

# Label suspicious images
dist = []
mask_pixels = []
masks = []
for id in df_train.index:
    img = cv2.imread(os.path.join('../data/train/images/','{}.png'.format(id)), cv2.IMREAD_GRAYSCALE)
    mask = cv2.imread(os.path.join('../data/train/masks/','{}.png'.format(id)), cv2.IMREAD_GRAYSCALE)
    dist.append(np.unique(img).shape[0])
    mask_pixels.append((mask/255).sum())
    masks.append(np.array(mask)/255.)
df_train['unique_pixels'] = dist
df_train['mask_pixels'] = mask_pixels
df_train['masks'] = masks

In [4]:
df_train.head()

,rle_mask,z,unique_pixels,mask_pixels,masks
id,,,,,
575d24d81d,NaN,843,225,0.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
a266a2a9df,5051 5151,794,238,5151.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
75efad62c1,9 93 109 94 210 94 310 95 411 95 511 96 612 96...,468,122,10133.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
34e51dba6a,48 54 149 54 251 53 353 52 455 51 557 50 659 4...,727,216,1522.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4875705fb0,1111 1 1212 1 1313 1 1414 1 1514 2 1615 2 1716...,797,219,437.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [5]:
#### Reference  from Heng's discussion
# https://www.kaggle.com/c/tgs-salt-identification-challenge/discussion/63984#382657
def get_mask_type(mask):
    border = 10
    outer = np.zeros((101-2*border, 101-2*border), np.float32)
    outer = cv2.copyMakeBorder(outer, border, border, border, border, borderType = cv2.BORDER_CONSTANT, value = 1)

    cover = (mask>0.5).sum()
    if cover == 0:
        return 0 # empty
    if cover == ((mask*outer) > 0.5).sum():
        return 1 #border
    if np.all(mask==mask[0]):
        return 2 #vertical

    percentage = cover/(101*101)
    if percentage < 0.15:
        return 3
    elif percentage < 0.25:
        return 4
    elif percentage < 0.50:
        return 5
    elif percentage < 0.75:
        return 6
    else:
        return 7

def histcoverage(coverage):
    histall = np.zeros((1,8))
    for c in coverage:
        histall[0,c] += 1
    return histall

df_train["coverage"] = df_train.masks.map(np.sum) / pow(101, 2)

df_train["coverage_class"] = df_train.masks.map(get_mask_type)

In [6]:
df_train.head()

,rle_mask,z,unique_pixels,mask_pixels,masks,coverage,coverage_class
id,,,,,,,
575d24d81d,NaN,843,225,0.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.000000,0
a266a2a9df,5051 5151,794,238,5151.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.504950,2
75efad62c1,9 93 109 94 210 94 310 95 411 95 511 96 612 96...,468,122,10133.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.993334,7
34e51dba6a,48 54 149 54 251 53 353 52 455 51 557 50 659 4...,727,216,1522.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.149201,3
4875705fb0,1111 1 1212 1 1313 1 1414 1 1514 2 1615 2 1716...,797,219,437.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.042839,1


In [7]:
df_train.coverage_class.value_counts()

0    1562
5     509
3     467
7     439
6     418
1     275
4     224
2     106
Name: coverage_class, dtype: int64

In [8]:
pd.crosstab(df_train.unique_pixels == 1,pd.isnull(df_train.rle_mask))

rle_mask,False,True
unique_pixels,,
False,2438,1482
True,0,80


In [9]:
df_train = df_train[df_train.unique_pixels > 1].sort_values(['mask_pixels','z'])
df_train.shape

(3920, 7)

In [10]:
puzzle = pd.read_csv('../puzzle_all.csv')
puzzle = puzzle[['id','mosaic_id','left','right','up','down','n_images']].set_index('id')

In [11]:
puzzle.head()

,mosaic_id,left,right,up,down,n_images
id,,,,,,
dd5a90cf89,1,cd8ed0c05d,14ab406aff,NaN,e34c1220b1,96
22a4bbce52,1,NaN,88b8522325,NaN,07244009e3,96
88b8522325,1,22a4bbce52,cd8ed0c05d,NaN,a0b00e7922,96
cd8ed0c05d,1,88b8522325,dd5a90cf89,NaN,fb903c3b68,96
e34c1220b1,1,fb903c3b68,6e9aceff2f,dd5a90cf89,c7a2535ee6,96


In [12]:
df_train.head()

,rle_mask,z,unique_pixels,mask_pixels,masks,coverage,coverage_class
id,,,,,,,
ad2113ea1c,NaN,54,152,0.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.0,0
eab589e1f2,NaN,63,194,0.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.0,0
dc534f1e28,NaN,63,195,0.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.0,0
022b1b01be,NaN,64,195,0.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.0,0
fcf61d1883,NaN,71,190,0.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.0,0


In [13]:
train_puzzle = df_train[['z','unique_pixels','coverage_class']].join(puzzle,how='left')

In [14]:
for col in ['left','right','up','down']:
    train_puzzle[col] = np.where(train_puzzle[col].isin(df_train.index),train_puzzle[col],None)

In [15]:
train_puzzle[pd.isnull(train_puzzle.mosaic_id)]

,z,unique_pixels,coverage_class,mosaic_id,left,right,up,down,n_images
id,,,,,,,,,
b0a7056a04,512,158,7,NaN,None,None,None,None,NaN


In [16]:
train_puzzle['mosaic_id'] = train_puzzle['mosaic_id'].fillna(max(train_puzzle['mosaic_id'])+1)

In [17]:
train_puzzle[pd.notnull(train_puzzle.left) | pd.notnull(train_puzzle.right) | pd.notnull(train_puzzle.up) | pd.notnull(train_puzzle.down)]

,z,unique_pixels,coverage_class,mosaic_id,left,right,up,down,n_images
id,,,,,,,,,
367c6fb412,83,241,0,17.0,None,None,74ab2a9b1a,None,71.0
f8ba90c991,88,250,0,2.0,None,59dabe134f,None,1986fcadda,94.0
77ecd76754,99,226,0,57.0,None,None,b2d7897c35,None,32.0
b1ac5ed0b3,99,210,0,20.0,None,ab7d09443f,None,None,68.0
dd9a94a797,102,207,0,79.0,fb47e8e74e,None,None,None,20.0
d538594e14,107,245,0,28.0,None,36b768bfa7,ef58213c9b,None,55.0
b2d7897c35,107,224,0,57.0,None,None,None,77ecd76754,32.0
a5d2da8488,108,232,0,1.0,None,None,917ef84e64,None,96.0
9ccf50ab0e,110,249,0,8.0,None,None,None,a1b5786252,83.0


In [18]:
train_puzzle['n_train_images'] = train_puzzle.groupby('mosaic_id').z.transform('count')
train_puzzle['mosaic_coverage_class'] = train_puzzle.groupby('mosaic_id').coverage_class.transform('mean')

In [20]:
train_puzzle.reset_index(inplace=True)

In [21]:
train_puzzle.head()

,id,z,unique_pixels,coverage_class,mosaic_id,left,right,up,down,n_images,n_train_images,mosaic_coverage_class
0,ad2113ea1c,54,152,0,901.0,None,None,None,None,1.0,1,0.0
1,eab589e1f2,63,194,0,512.0,None,None,None,None,1.0,1,0.0
2,dc534f1e28,63,195,0,2728.0,None,None,None,None,1.0,1,0.0
3,022b1b01be,64,195,0,1955.0,None,None,None,None,1.0,1,0.0
4,fcf61d1883,71,190,0,3019.0,None,None,None,None,1.0,1,0.0


In [ ]:


id1 = '1306fcee4c'
id2 = '04182ced8e'

In [ ]:
# a=cv2.imread('../data/train/images/{}.png'.format(id1),0)
# b=cv2.imread('../data/train/images/{}.png'.format(id2),0)

# plt.figure(figsize=(3,4))
# plt.imshow(a,cmap='gray')

# plt.figure(figsize=(3,4))
# plt.imshow(b,cmap='gray')


In [ ]:
# a=cv2.imread('../data/train/masks/{}.png'.format(id1),0)
# b=cv2.imread('../data/train/masks/{}.png'.format(id2),0)

# plt.figure(figsize=(3,4))
# plt.imshow(a,cmap='gray')

# plt.figure(figsize=(3,4))
# plt.imshow(b,cmap='gray')


In [22]:
train_puzzle.shape

(3920, 12)

In [23]:
train_puzzle_folds = train_puzzle.reset_index()[['mosaic_id','mosaic_coverage_class','n_train_images']].drop_duplicates().sample(frac=1,random_state=13)

In [24]:
train_puzzle_folds = train_puzzle_folds.sort_values(['n_train_images','mosaic_coverage_class'])

In [25]:
n_folds = 10
train_puzzle_folds['fold'] = (list(range(n_folds)) * train_puzzle_folds.shape[0])[:train_puzzle_folds.shape[0]]
train_puzzle_folds['fold'].value_counts()

9    315
7    315
5    315
3    315
1    315
8    315
6    315
4    315
2    315
0    315
Name: fold, dtype: int64

In [26]:
train_puzzle_folds.head(20)

,mosaic_id,mosaic_coverage_class,n_train_images,fold
30,3222.0,0.0,1,0
984,2918.0,0.0,1,1
1398,1705.0,0.0,1,2
504,2087.0,0.0,1,3
1231,2771.0,0.0,1,4
445,775.0,0.0,1,5
1101,2601.0,0.0,1,6
1261,1140.0,0.0,1,7
1209,1296.0,0.0,1,8
977,651.0,0.0,1,9


In [27]:
train_puzzle_folds.shape

(3150, 4)

In [28]:
train_puzzle = train_puzzle.merge(train_puzzle_folds)

In [29]:
train_puzzle

,id,z,unique_pixels,coverage_class,mosaic_id,left,right,up,down,n_images,n_train_images,mosaic_coverage_class,fold
0,ad2113ea1c,54,152,0,901.0,None,None,None,None,1.0,1,0.000000,7
1,eab589e1f2,63,194,0,512.0,None,None,None,None,1.0,1,0.000000,9
2,dc534f1e28,63,195,0,2728.0,None,None,None,None,1.0,1,0.000000,2
3,022b1b01be,64,195,0,1955.0,None,None,None,None,1.0,1,0.000000,0
4,fcf61d1883,71,190,0,3019.0,None,None,None,None,1.0,1,0.000000,2
5,4c36178b41,72,192,0,1736.0,None,None,None,None,1.0,1,0.000000,3
6,df5df272fb,73,185,0,1299.0,None,None,None,None,1.0,1,0.000000,0
7,44381a3f55,73,110,0,2173.0,None,None,None,None,1.0,1,0.000000,9
8,9a48617407,74,120,0,483.0,None,None,None,None,1.0,1,0.000000,8
9,d18a64f984,76,212,0,2032.0,None,None,None,None,1.0,1,0.000000,3


In [30]:
train_puzzle.fold.value_counts()

9    406
7    397
8    397
6    394
5    393
4    389
3    388
1    386
2    386
0    384
Name: fold, dtype: int64

In [35]:
train_puzzle[['id','fold','coverage_class','left','right','up','down']].to_csv(os.path.join(DATA_ROOT,'train_proc_puzzle.csv'),index=False)

In [36]:
train_puzzle.shape

(3920, 13)

In [38]:
train_puzzle[['id','fold','coverage_class','left','right','up','down']].tail()

,id,fold,coverage_class,left,right,up,down
3915,da082ec985,4,7,None,None,None,None
3916,ae51614513,2,7,None,None,None,None
3917,58ad72a266,1,7,None,None,None,None
3918,56b5540fd7,0,7,None,None,None,None
3919,ef13247eed,4,7,None,None,None,None


In [39]:
train_puzzle[pd.notnull(train_puzzle.left) | pd.notnull(train_puzzle.right) | pd.notnull(train_puzzle.up) | pd.notnull(train_puzzle.down)]

,id,z,unique_pixels,coverage_class,mosaic_id,left,right,up,down,n_images,n_train_images,mosaic_coverage_class,fold
16,0429d9759b,737,193,6,30.0,None,None,None,f7380099f6,52.0,9,1.111111,1
18,f7380099f6,796,220,2,30.0,None,None,0429d9759b,None,52.0,9,1.111111,1
20,1b9865501f,180,233,0,63.0,None,None,d4e9588d7c,None,27.0,9,2.111111,2
21,d4e9588d7c,377,227,0,63.0,None,9c5c26e2ba,ca0d3ea2f5,1b9865501f,27.0,9,2.111111,2
22,77608c7770,512,160,0,63.0,None,None,None,548887f17d,27.0,9,2.111111,2
23,9c5c26e2ba,639,213,0,63.0,d4e9588d7c,None,None,None,27.0,9,2.111111,2
24,547c9d83fd,206,212,3,63.0,None,None,None,ca0d3ea2f5,27.0,9,2.111111,2
25,548887f17d,525,155,4,63.0,None,None,77608c7770,None,27.0,9,2.111111,2
27,ca0d3ea2f5,637,201,7,63.0,None,None,547c9d83fd,d4e9588d7c,27.0,9,2.111111,2
29,367c6fb412,83,241,0,17.0,None,None,74ab2a9b1a,None,71.0,15,0.000000,8
